GBIF PREPROCESSING, SPLITTING + BY TAXONKEY

In [1]:
import numpy as np
import pandas as pd
import os, os.path

# Column selection

In [2]:
selected_columns = ['countryCode','decimalLatitude','decimalLongitude','year','taxonKey','basisOfRecord','institutionCode','recordedBy']
selected_column_numbers = [14,17,18,28,29,31,32,40]
column_dtypes = ['object','float32','float32','float32','uint32','object','object','object']

In [6]:
path = '/home/joon/data/'
name = "gbif-"+'-'.join(selected_columns)

In [7]:
# Sélection des colonnes
os.system("cut -d'	' -f"+",".join(map(str,selected_column_numbers)) +" /home/joon/data/gbif-complet.csv > /home/joon/data/gbif-"+'-'.join(selected_columns)+".csv")

0

In [8]:
os.system("sed 's/\t\t/\t0\t/g' /home/joon/data/gbif-"+'-'.join(selected_columns)+".csv > /home/joon/data/tmp.csv")

0

In [9]:
os.system("sed 's/\t\t/\t0\t/g' /home/joon/data/tmp.csv > /home/joon/data/tmp2.csv")
os.system("rm /home/joon/data/tmp.csv")

0

In [10]:
os.system("sed 's/\"//g' /home/joon/data/tmp2.csv > /home/joon/data/gbif-"+'-'.join(selected_columns)+".csv")
os.system("rm /home/joon/data/tmp2.csv")

0

# Splitting

In [11]:
os.system("mkdir /home/joon/data/gbif-"+'-'.join(selected_columns)+"-splits")

0

In [12]:
os.system("split -a4 -d -l100000 /home/joon/data/gbif-"+'-'.join(selected_columns)+".csv /home/joon/data/gbif-"+'-'.join(selected_columns)+"-splits/")

0

In [13]:
# Remove header in first split
os.system("tail -n +2 "+path+name+'-splits/0000 > '+path+name+'-splits/tmp && '+"mv "+path+name+'-splits/tmp '+path+name+'-splits/0000')

0

# By taxonkey by split

In [14]:
# Rq: les splits commencent au numéro 0
n_splits = len(os.listdir(path+name+'-splits'))

In [15]:
os.system('mkdir '+path+name+'-by-taxonkey/')

0

In [16]:
being_processed = []
def process_split(split_nb):
    global being_processed
    being_processed.append(split_nb)
    os.system('mkdir '+path+name+'-by-taxonkey/split-'+str(split_nb).zfill(4))
    gbif_distribution_split = pd.read_csv(path+name+'-splits/'+str(split_nb).zfill(4),
                                          sep='	',
                                          index_col=selected_columns.index('taxonKey'),
                                          header=None,
                                          dtype=dict(zip(range(len(selected_columns)),column_dtypes)),
                                          engine='python',
                                          quoting=3)
    taxonKey_values = gbif_distribution_split.index.unique()
    for taxonKey in taxonKey_values:
        gbif_distribution_split.loc[[taxonKey]].to_csv(path+name+'-by-taxonkey/split-'+str(split_nb).zfill(4)+'/'+str(taxonKey)+'.csv',header=False,index=False,sep=',')
    being_processed.remove(split_nb)

In [17]:
from multiprocessing import Pool
with Pool(processes=11) as pool:
    pool.map(process_split, range(n_splits))

# Fusion files

In [18]:
import glob, re
list_files = glob.glob(path+name+'-by-taxonkey/split-*/*.csv')
list_files_wo_path = list(map(lambda x: re.sub(path+name+'-by-taxonkey/split-[0-9]+/','',x),list_files))
list_files_wo_path = list(map(lambda x:re.sub('.csv','',x),list_files_wo_path))
list_files_wo_path = list(set(list_files_wo_path))

In [19]:
os.system('mkdir '+path+name+'-by-taxonkey-fusionned/')
for taxonKey in list_files_wo_path:
    os.system('cat '+path+name+'-by-taxonkey/split-*/'+taxonKey+'.csv > '+path+name+'-by-taxonkey-fusionned/'+taxonKey+'.csv')

In [22]:
os.system('rm -r '+path+name+'-by-taxonkey')
os.system('rm -r '+path+name+'-splits')

0

# Basis of record 

In [4]:
selected_columns = ['basisOfRecord']
selected_column_numbers = [31]

In [5]:
path = '/home/joon/data/'
name = "gbif-"+'-'.join(selected_columns)

In [6]:
# Sélection des colonnes
os.system("cut -d'	' -f"+",".join(map(str,selected_column_numbers)) +" /home/joon/data/gbif-complet.csv > /home/joon/data/gbif-"+'-'.join(selected_columns)+".csv")

0

In [7]:
basisOfRecord = pd.read_csv("/home/joon/data/gbif-"+'-'.join(selected_columns)+".csv",sep='	',header=None,index_col=None)

In [30]:
basisOfRecord_values = basisOfRecord.loc[1:,0].unique()

In [33]:
pd.Series(basisOfRecord_values).to_csv('/home/joon/data/gbif-basisOfRecord-values.csv')

<ipython-input-33-b17bdf5b3ca0>:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  pd.Series(basisOfRecord_values).to_csv('/home/joon/data/gbif-basisOfRecord-values.csv')
